In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import math 


In [2]:
data = pd.read_csv(r"C:\Users\jacar\OneDrive\Desktop\Code\GC360KNNIDEA.csv")
nax = 797
df = pd.DataFrame(data)
df

,Trade #,Algo,Entry Date,Side,Symbol,Entry,Exit,P/L,Unnamed: 8
0,1,NaN,NaN,long,GC,20021015,20021015,0,-740.0
1,2,NaN,NaN,NaN,NaN,20021016,20021017,10,NaN
2,3,NaN,NaN,NaN,NaN,20021024,20021024,-80,NaN
3,4,NaN,NaN,NaN,NaN,20021031,20021031,-40,NaN
4,5,NaN,NaN,NaN,NaN,20021107,20021107,30,NaN
...,...,...,...,...,...,...,...,...,...
792,793,NaN,NaN,NaN,NaN,20180409,20180410,290,NaN
793,794,NaN,NaN,NaN,NaN,20180417,20180417,240,NaN
794,795,NaN,NaN,NaN,NaN,20180420,20180423,-400,NaN
795,796,NaN,NaN,NaN,NaN,20180426,20180426,-680,NaN


In [3]:
df = df.drop('Trade #',axis=1)
df = df.drop('Entry Date',axis=1)
df = df.drop('Unnamed: 8',axis=1)
df = df.drop('Algo',axis=1)
df = df.drop('Side',axis=1)
df = df.drop('Symbol',axis=1)

df

,Entry,Exit,P/L
0,20021015,20021015,0
1,20021016,20021017,10
2,20021024,20021024,-80
3,20021031,20021031,-40
4,20021107,20021107,30
...,...,...,...
792,20180409,20180410,290
793,20180417,20180417,240
794,20180420,20180423,-400
795,20180426,20180426,-680


In [4]:
#lst = {'Quartz':0, 'CAS':1, 'iii':2, 'Kyanite':3, 'tday':4, 'SOJ':5, 'tday ':6, 'voly':7, 'lux':8, 'milli2':9, 'lux ':10}
#lst2 = {'long':0, 'short':1, 'short ':1}
#lst3 = {'GC':0, 'ES':1, 'euro':2, 'euro ':3, 'EC':4, 'CLN21':5, 'S':6, 'CLU21':7, 'ES ':8,
       #'W':9, 'CLV21':10, 'CLX21':11, 'ECZ21':12, 'ESZ21':13, 'GCZ21':14, 'WZ21':15, 'SF22':16,
       #'GCG22':17, 'ECH22':18, 'CLF22':19, 'ESH22':20, 'CLG22':21, 'SH22':22, 'CLZ21':23,
       #'WH22':24}
#df['Algo'] = df['Algo'].map(lst)
#df['Side'] = df['Side'].map(lst2)
#df['Symbol'] = df['Symbol'].map(lst3)



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Entry   797 non-null    int64
 1   Exit    797 non-null    int64
 2   P/L     797 non-null    int64
dtypes: int64(3)
memory usage: 18.8 KB


In [6]:
# For dataset normalization
tot_vm2 = 0
for i in range(0,2): #5
  mean2 = df.iloc[:,i].mean()
  for j in range(0,2):
    v_m2 = (df.iloc[j,i] - mean2)**2
    #print(v_m)
    tot_vm2 += v_m2 
  std2 = math.sqrt(tot_vm2/2)
  for k in range(0,2):
    nor2 = (df.iloc[k,i] - mean2)/std2 
    df.iloc[k,i] = nor2 
print("Data Normalization Done!...")

Data Normalization Done!...


In [7]:
import math

In [8]:
for i in range(0,nax):
  if(df.iloc[i,2] < 0):
    df.iloc[i,2] = 0   ########## looser
  else:
    df.iloc[i,2] = 1   ########## winner

In [9]:
df = df.sample(frac = 1)
df

,Entry,Exit,P/L
62,20040202.0,20040202.0,0
166,20060313.0,20060313.0,1
99,20041210.0,20041213.0,1
597,20140611.0,20140611.0,1
286,20080808.0,20080808.0,0
...,...,...,...
128,20050720.0,20050721.0,1
686,20151211.0,20151211.0,0
585,20140327.0,20140327.0,0
120,20050526.0,20050526.0,0


In [10]:
def knn_train_fun(df):
  K=3
  accu=0
  dis = []
  indx = []
  aa='aa'
  for i in range(1,nax): ########## 143
    dis = []
    indx = []
    aa='aa'
    row1 = df.iloc[i,:]    # here i am taking test sample one by one
    aa = row1[2] 
    #print(row1)

    # HEre i am iterating test sample over all the training samples
    # and calculating L2 distance and storing in dis list
    for m in range(0,nax):
        dis_tot = 0
        row2 = df.iloc[m,:]
        for j in range(0,3):
          dis_1point = (row1[j] - row2[j])**2
          dis_tot += dis_1point
        dis_tot = math.sqrt(dis_tot)
        dis.append(dis_tot)
    # here i am getting the k-minimum distance index numbers to find their class or label
    for j in range(0,K):
      fst_min =  dis.index(min(dis))
      dis[fst_min] = 99999
      indx.append(fst_min)
    # here i am getting the class of the minimum distances and assigning the calss to the majority count
    maj = []
    for j in range(0,K):
      maj.append(df.iloc[indx[j],2])
    #print(maj)
    cunt = Counter(maj).values() 
    cunt = list(cunt)
    class_index = cunt.index(max(cunt))
    keey = Counter(maj).keys()
    keey = list(keey)
    pred_class = keey[class_index]
    #print('actual: ',aa)
    #print('predic: ',pred_class)
    bb=0
    if ( aa == pred_class):
      accu+=1
      bb=1
    print("ID=",i,' Predicted= ',pred_class,' True= ',aa,'Accuracy: ',bb)
    bb= 0

  classification_accuracy = accu/nax
  print("Training classification accuracy= ", classification_accuracy);


knn_train_fun(df)

ID= 1  Predicted=  0  True=  1.0 Accuracy:  0
ID= 2  Predicted=  1  True=  1.0 Accuracy:  1
ID= 3  Predicted=  1  True=  1.0 Accuracy:  1
ID= 4  Predicted=  0  True=  0.0 Accuracy:  1
ID= 5  Predicted=  1  True=  1.0 Accuracy:  1
ID= 6  Predicted=  0  True=  0.0 Accuracy:  1
ID= 7  Predicted=  0  True=  0.0 Accuracy:  1
ID= 8  Predicted=  1  True=  1.0 Accuracy:  1
ID= 9  Predicted=  1  True=  1.0 Accuracy:  1
ID= 10  Predicted=  0  True=  0.0 Accuracy:  1
ID= 11  Predicted=  1  True=  0.0 Accuracy:  0
ID= 12  Predicted=  1  True=  1.0 Accuracy:  1
ID= 13  Predicted=  1  True=  1.0 Accuracy:  1
ID= 14  Predicted=  1  True=  1.0 Accuracy:  1
ID= 15  Predicted=  1  True=  0.0 Accuracy:  0
ID= 16  Predicted=  0  True=  0.0 Accuracy:  1
ID= 17  Predicted=  0  True=  0.0 Accuracy:  1
ID= 18  Predicted=  1  True=  0.0 Accuracy:  0
ID= 19  Predicted=  0  True=  1.0 Accuracy:  0
ID= 20  Predicted=  1  True=  1.0 Accuracy:  1
ID= 21  Predicted=  1  True=  1.0 Accuracy:  1
ID= 22  Predicted=  1 

ID= 174  Predicted=  1  True=  0.0 Accuracy:  0
ID= 175  Predicted=  1  True=  1.0 Accuracy:  1
ID= 176  Predicted=  0  True=  1.0 Accuracy:  0
ID= 177  Predicted=  0  True=  0.0 Accuracy:  1
ID= 178  Predicted=  1  True=  1.0 Accuracy:  1
ID= 179  Predicted=  1  True=  1.0 Accuracy:  1
ID= 180  Predicted=  0  True=  0.0 Accuracy:  1
ID= 181  Predicted=  0  True=  1.0 Accuracy:  0
ID= 182  Predicted=  1  True=  0.0 Accuracy:  0
ID= 183  Predicted=  1  True=  0.0 Accuracy:  0
ID= 184  Predicted=  1  True=  0.0 Accuracy:  0
ID= 185  Predicted=  1  True=  1.0 Accuracy:  1
ID= 186  Predicted=  1  True=  1.0 Accuracy:  1
ID= 187  Predicted=  0  True=  1.0 Accuracy:  0
ID= 188  Predicted=  1  True=  1.0 Accuracy:  1
ID= 189  Predicted=  1  True=  1.0 Accuracy:  1
ID= 190  Predicted=  0  True=  0.0 Accuracy:  1
ID= 191  Predicted=  1  True=  0.0 Accuracy:  0
ID= 192  Predicted=  1  True=  1.0 Accuracy:  1
ID= 193  Predicted=  1  True=  1.0 Accuracy:  1
ID= 194  Predicted=  0  True=  0.0 Accur

ID= 345  Predicted=  1  True=  0.0 Accuracy:  0
ID= 346  Predicted=  1  True=  1.0 Accuracy:  1
ID= 347  Predicted=  1  True=  1.0 Accuracy:  1
ID= 348  Predicted=  1  True=  1.0 Accuracy:  1
ID= 349  Predicted=  0  True=  1.0 Accuracy:  0
ID= 350  Predicted=  1  True=  0.0 Accuracy:  0
ID= 351  Predicted=  1  True=  1.0 Accuracy:  1
ID= 352  Predicted=  0  True=  1.0 Accuracy:  0
ID= 353  Predicted=  0  True=  1.0 Accuracy:  0
ID= 354  Predicted=  0  True=  0.0 Accuracy:  1
ID= 355  Predicted=  1  True=  1.0 Accuracy:  1
ID= 356  Predicted=  0  True=  0.0 Accuracy:  1
ID= 357  Predicted=  0  True=  1.0 Accuracy:  0
ID= 358  Predicted=  1  True=  1.0 Accuracy:  1
ID= 359  Predicted=  1  True=  1.0 Accuracy:  1
ID= 360  Predicted=  1  True=  1.0 Accuracy:  1
ID= 361  Predicted=  1  True=  1.0 Accuracy:  1
ID= 362  Predicted=  1  True=  1.0 Accuracy:  1
ID= 363  Predicted=  1  True=  1.0 Accuracy:  1
ID= 364  Predicted=  1  True=  1.0 Accuracy:  1
ID= 365  Predicted=  1  True=  1.0 Accur

ID= 516  Predicted=  1  True=  1.0 Accuracy:  1
ID= 517  Predicted=  1  True=  0.0 Accuracy:  0
ID= 518  Predicted=  0  True=  1.0 Accuracy:  0
ID= 519  Predicted=  1  True=  1.0 Accuracy:  1
ID= 520  Predicted=  1  True=  1.0 Accuracy:  1
ID= 521  Predicted=  0  True=  0.0 Accuracy:  1
ID= 522  Predicted=  0  True=  0.0 Accuracy:  1
ID= 523  Predicted=  1  True=  0.0 Accuracy:  0
ID= 524  Predicted=  0  True=  0.0 Accuracy:  1
ID= 525  Predicted=  1  True=  1.0 Accuracy:  1
ID= 526  Predicted=  0  True=  0.0 Accuracy:  1
ID= 527  Predicted=  0  True=  0.0 Accuracy:  1
ID= 528  Predicted=  0  True=  0.0 Accuracy:  1
ID= 529  Predicted=  1  True=  1.0 Accuracy:  1
ID= 530  Predicted=  1  True=  0.0 Accuracy:  0
ID= 531  Predicted=  1  True=  1.0 Accuracy:  1
ID= 532  Predicted=  0  True=  0.0 Accuracy:  1
ID= 533  Predicted=  1  True=  1.0 Accuracy:  1
ID= 534  Predicted=  0  True=  0.0 Accuracy:  1
ID= 535  Predicted=  0  True=  1.0 Accuracy:  0
ID= 536  Predicted=  1  True=  1.0 Accur

ID= 687  Predicted=  1  True=  0.0 Accuracy:  0
ID= 688  Predicted=  0  True=  0.0 Accuracy:  1
ID= 689  Predicted=  1  True=  1.0 Accuracy:  1
ID= 690  Predicted=  0  True=  0.0 Accuracy:  1
ID= 691  Predicted=  0  True=  1.0 Accuracy:  0
ID= 692  Predicted=  1  True=  0.0 Accuracy:  0
ID= 693  Predicted=  0  True=  0.0 Accuracy:  1
ID= 694  Predicted=  1  True=  0.0 Accuracy:  0
ID= 695  Predicted=  1  True=  1.0 Accuracy:  1
ID= 696  Predicted=  1  True=  1.0 Accuracy:  1
ID= 697  Predicted=  0  True=  0.0 Accuracy:  1
ID= 698  Predicted=  0  True=  0.0 Accuracy:  1
ID= 699  Predicted=  1  True=  0.0 Accuracy:  0
ID= 700  Predicted=  1  True=  1.0 Accuracy:  1
ID= 701  Predicted=  1  True=  1.0 Accuracy:  1
ID= 702  Predicted=  1  True=  1.0 Accuracy:  1
ID= 703  Predicted=  1  True=  1.0 Accuracy:  1
ID= 704  Predicted=  1  True=  0.0 Accuracy:  0
ID= 705  Predicted=  1  True=  0.0 Accuracy:  0
ID= 706  Predicted=  1  True=  1.0 Accuracy:  1
ID= 707  Predicted=  1  True=  1.0 Accur

In [11]:
############# Test Unknow Sample


In [12]:
def knn_test_fun(test,df):
  K=3
  accu=0
  dis = []
  indx = []
  row1 = test    # here i am taking test sample one by one

  # HEre i am iterating test sample over all the training samples
  # and calculating L2 distance and storing in dis list
  for m in range(0,nax):
      dis_tot = 0
      row2 = df.iloc[m,:]
      for j in range(0,2):
        dis_1point = (row1[j] - row2[j])**2
        dis_tot += dis_1point
      dis_tot = math.sqrt(dis_tot)
      dis.append(dis_tot)
  # here i am getting the k-minimum distance index numbers to find their class or label
  for j in range(0,K):
    fst_min =  dis.index(min(dis))
    dis[fst_min] = 99999
    indx.append(fst_min)
  # here i am getting the class of the minimum distances and assigning the calss to the majority count
  maj = []
  for j in range(0,K):
    maj.append(df.iloc[indx[j],2])
  #print(maj)
  cunt = Counter(maj).values() 
  cunt = list(cunt)
  class_index = cunt.index(max(cunt))
  keey = Counter(maj).keys()
  keey = list(keey)
  pred_class = keey[class_index]
  #print('actual: ',aa)
  #print('predic: ',pred_class)
  print('Your test data is Predicted as = ',pred_class)



In [13]:
###########################
# 0 = Looser ,1 = Winner
###########################
test = [20021015	,20021015]
knn_test_fun(test,df)

Your test data is Predicted as =  0
